# Competitor Analysis

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# import data

candidates_rt = pd.read_csv('../data/temproary_data/candidate_roundTrip_route.csv')
candidates_rt['round_trip_route_IATA'] = candidates_rt['round_trip_route_IATA'].apply(eval)
roundTrips = pd.read_csv('../data/temproary_data/round_trip_flights.csv')
roundTrips['round_trip_route_IATA'] = roundTrips['round_trip_route_IATA'].apply(eval)
roundTrips['inbound_FL_DATE'] = pd.to_datetime(roundTrips['inbound_FL_DATE'])
roundTrips['outbound_FL_DATE'] = pd.to_datetime(roundTrips['outbound_FL_DATE'])
airports_info = pd.read_csv('../data/cleaned_data/Airport_Codes.csv')
tickets_info = pd.read_csv('../data/cleaned_data/Tickets.csv')


tickets_info = tickets_info.assign(
        sorted_route=tickets_info.apply(
            lambda x: tuple(
                sorted([x["ORIGIN_AIRPORT_IATA_CODE"], x["DEST_AIRPORT_IATA_CODE"]])
            ),
            axis=1,
        )
    )

round_trip_profit = pd.read_csv('../data/temproary_data/roundTrip_profit.csv')
round_trip_profit['round_trip_route_IATA'] = round_trip_profit['round_trip_route_IATA'].apply(eval)
avg_ticket_price = pd.read_csv('../data/temproary_data/average_ticket_price.csv')
all_flights = pd.read_csv('../data/original_data/Flights.csv')


# gather relevent data

airports_can = {i for pair in candidates_rt['round_trip_route_IATA'].values for i in pair}

candidate_roundTrips = roundTrips[roundTrips['round_trip_route_IATA'].isin(candidates_rt['round_trip_route_IATA'])]
candidate_airports = airports_info[airports_info['AIRPORT_IATA_CODE'].apply(lambda x: x in airports_can)]
candidate_tickets = tickets_info[tickets_info['sorted_route'].isin(candidates_rt['round_trip_route_IATA'])]

/opt/anaconda3/envs/tongConsultinInc/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3508: DtypeWarning: Columns (3,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Competitor associatd with each candidate round trip route

In [53]:
# find the operation carieer assoicated with the top 10 around trip route
roundTrip_op = candidate_roundTrips.groupby('round_trip_route_IATA')['inbound_OP_CARRIER'].apply(lambda x: set(x.value_counts().index)).reset_index()
roundTrip_op['op_count'] = roundTrip_op['inbound_OP_CARRIER'].apply(len)

In [61]:
operating_carrier = pd.DataFrame(pd.Series(list(set([j for i in roundTrip_op['inbound_OP_CARRIER'].values for j in i])), name='OP_CARRIER'))


# the proportion of cancel flights associated with each operating carieer

In [100]:
all_flights['round_trip'] = all_flights.apply(lambda row : tuple(sorted([row['ORIGIN'], row['DESTINATION']])), axis=1)

In [116]:
op_prop_cancel = all_flights[(all_flights['round_trip'].isin(candidates_rt['round_trip_route_IATA'])) & (all_flights['OP_CARRIER'].isin(operating_carrier['OP_CARRIER']))].groupby(['OP_CARRIER', 'round_trip'])['CANCELLED'].mean().reset_index().rename(columns={'CANCELLED': 'prop_cancel'})

In [124]:
op_prop_cancel.to_csv('../data/temproary_data/op_prop_cancel.csv', index=False)

# Average delay rate assoicated with each operating carieer

In [118]:
all_flights = pd.concat(
    [candidate_roundTrips[['inbound_DEP_DELAY', 'inbound_ARR_DELAY', 'inbound_OP_CARRIER' ,'round_trip_route_IATA']].rename(columns={'inbound_DEP_DELAY':'DEP_DELAY', 'inbound_ARR_DELAY':'ARR_DELAY', 'inbound_OP_CARRIER': 'OP_CARRIER'}, inplace= False),
    candidate_roundTrips[['outbound_DEP_DELAY', 'outbound_ARR_DELAY', 'outbound_OP_CARRIER' ,'round_trip_route_IATA']].rename(columns={'outbound_DEP_DELAY':'DEP_DELAY', 'outbound_ARR_DELAY':'ARR_DELAY', 'outbound_OP_CARRIER': 'OP_CARRIER'}, inplace= False)],
    axis=0,
    ignore_index=True
)

In [123]:
all_flights.groupby(['OP_CARRIER', 'round_trip_route_IATA'])['DEP_DELAY', 'ARR_DELAY'].mean().reset_index().to_csv('../data/temproary_data/op_average_delay.csv', index=False)

/var/folders/0y/gbwmzjp93k12t06yhk8_2p7h0000gn/T/ipykernel_34231/2552485323.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  all_flights.groupby(['OP_CARRIER', 'round_trip_route_IATA'])['DEP_DELAY', 'ARR_DELAY'].mean().reset_index().to_csv('../data/temproary_data/op_average_delay.csv', index=False)
